In [1]:
import torch

Comprobando el padding

In [56]:
# (batch, seq_length, max_patch) = (batch, past_time, tgt)
#past_observed_target = torch.ones(1,4,2)
past_observed_target = torch.ones(5,500,50)
batch, past_time, tgt = past_observed_target.shape

In [58]:
past_observed_target.shape

torch.Size([5, 500, 50])

In [59]:
max_patch = 6
patch_size = 5
#patch_size = 7
pad = (-tgt % patch_size, 0)
print(pad)

(0, 0)


Nota: cuidado al interpretar el residuo con valores negativos

In [60]:
coef = -tgt/patch_size
print(coef) 
coef = round(coef)
print(coef) 
res = -tgt - coef*patch_size
print ( res )

-10.0
-10
0


In [61]:
round(res)

0

In [62]:
res = torch.nn.functional.pad(
    input = past_observed_target,
    pad   = pad,
    mode  = 'constant',
    value = None
)

In [63]:
res.shape

torch.Size([5, 500, 50])

In [24]:
res

tensor([[[0., 1., 1.],
         [0., 1., 1.],
         [0., 1., 1.],
         [0., 1., 1.]]])

Ha aumentado en 1 la tercera dimensión (la última), metiendo ceros por delante

In [42]:
from einops import reduce

In [43]:
past_seq_id = reduce(
    res, 
    "... (seq patch) dim -> ... seq",
    "max",
    patch = patch_size
)

EinopsError:  Error while processing max-reduction pattern "... (seq patch) dim -> ... seq".
 Input tensor shape: torch.Size([1, 4, 7]). Additional info: {'patch': 7}.
 Shape mismatch, can't divide axis of length 4 in chunks of 7

A juzgar por el error, va a tratar de dividir la segunda dimensión en lotes de patch_dim
Ajustemos los valores

In [48]:
# (batch, seq_length, max_patch) = (batch, past_time, tgt)
past_observed_target = torch.ones(5,500,50)
batch, past_time, tgt = past_observed_target.shape
print (past_observed_target.shape)
max_patch = 6
patch_size = 5
pad = (-tgt % patch_size, 0)
print(pad)

res = torch.nn.functional.pad(
    input = past_observed_target,
    pad   = pad,
    mode  = 'constant',
    value = None
)

print (res.shape)

past_seq_id = reduce(
    res, 
    "... (seq patch) dim -> ... seq",
    "max",
    patch = patch_size
)

torch.Size([5, 500, 50])
(0, 0)
torch.Size([5, 500, 50])


In [50]:
print(past_seq_id.shape)

torch.Size([5, 100])


Resumen 

past_observed_target ~ (batch, past_time, tgt)
padded_tensor ~ (batch, past_time, padded_tgt) , donde en padded_tgt se ha ajustado tgt para ser multiplo de patch_size
past_seq_id ~ (batch, past_time, reduced_tgt)

In [69]:
import torch
from einops import reduce

# Supongamos que tenemos un tensor de tamaño (batch, past_time, tgt)
batch_size = 2
past_time = 8
tgt = 5
patch_size = 4

# Crear un tensor de ejemplo
past_observed_target = torch.randn(batch_size, past_time, tgt)

# Función de padding (ejemplo simplificado)
def _patched_seq_pad(patch_size, tensor, dim, left=True):
    size = tensor.size(dim)
    pad_length = -size % patch_size
    if pad_length > 0:
        pad = (pad_length, 0) if left else (0, pad_length)
        tensor = torch.nn.functional.pad(tensor, pad, "constant", 0)
    return tensor
print(f"observed: {past_observed_target.shape}")
# Aplicar padding
padded_tensor = _patched_seq_pad(patch_size, past_observed_target, -1, left=True)
print(f"Padded: {padded_tensor.shape}")
# Aplicar reducción con einops.reduce
past_seq_id = reduce(
    padded_tensor,
    "... (seq patch) dim -> ... seq",
    "max",
    patch=patch_size
)
print(f"Aplastando la última dimensión con máximo {past_seq_id.shape}")  # Salida: (batch_size, past_time//patch_size)

observed: torch.Size([2, 8, 5])
Padded: torch.Size([2, 8, 8])
Aplastando la última dimensión con máximo torch.Size([2, 2])


In [70]:
padded_tensor

tensor([[[ 0.0000,  0.0000,  0.0000, -0.6181, -0.3906, -1.2436,  1.1761,
          -0.1643],
         [ 0.0000,  0.0000,  0.0000,  1.8827,  0.5280,  0.7877,  0.5154,
           1.2527],
         [ 0.0000,  0.0000,  0.0000, -1.1373,  0.0117,  0.1479,  1.0995,
           0.3216],
         [ 0.0000,  0.0000,  0.0000, -2.0903, -0.9213, -0.2917,  1.6650,
          -0.1102],
         [ 0.0000,  0.0000,  0.0000, -0.1578,  0.4736, -1.4402, -0.2708,
          -0.9566],
         [ 0.0000,  0.0000,  0.0000, -1.0781,  0.8618, -0.4100,  2.8120,
          -0.1540],
         [ 0.0000,  0.0000,  0.0000,  1.3022, -0.6925,  0.1545,  0.3131,
          -1.1866],
         [ 0.0000,  0.0000,  0.0000,  0.9113, -0.2068, -0.5051, -1.5341,
          -0.0877]],

        [[ 0.0000,  0.0000,  0.0000,  0.0999, -1.7418, -0.9186,  1.3988,
           1.0960],
         [ 0.0000,  0.0000,  0.0000, -1.9745,  1.8979, -1.9370,  1.5482,
           1.0405],
         [ 0.0000,  0.0000,  0.0000,  0.1731, -2.0370,  0.7417, -0.6

In [71]:
past_seq_id

tensor([[1.8827, 2.8120],
        [1.8979, 2.2819]])